<a href="https://colab.research.google.com/github/ayoubajermoune/Windows-VPS/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [2]:
import requests

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import csv
import time


options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)
traders_list = []
def dianmic_webset():
    url_list = ['https://my.litefinance.org/traders?type=profit', 'https://my.litefinance.org/traders?type=risk']
    for url in url_list:
        browser.get(url)
        for _ in range(1):
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
        content = BeautifulSoup(browser.page_source , 'html.parser')
        traders = content.find_all('div' , {'class' : 'card_trader'})
        for trader in traders:
          profitability = trader.find('div' , {'class' : 'user_info'}).find('div' , {'class' : 'data'}).find('div' , {'class' : 'data_value'})
          profitability = profitability.text.strip().replace('%', '')
          if float(profitability) >= 50:
            user_info = trader.find('div' , {'class' : 'user_info'}).find('div' , {'class' : 'title'})
            id = trader.find('a' , {'class' : 'link'}).get('href')
            result = {
                'trader name': user_info.text.strip(),
                'id': id.strip().replace('/traders/info?id=', ''),
                'Profitability (All time) %' : profitability
            }
            traders_list.append(result)
    with open('traders.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['trader name', 'id' , 'Profitability (All time) %']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for trader in traders_list:
            writer.writerow(trader)
dianmic_webset()


In [4]:
def annual_rates(id:int,path:str):
    url_trader = f'https://my.litefinance.org/traders/info?id={id}'
    r = requests.get(url_trader)
    if r.status_code == 200:
        content = BeautifulSoup(r.text, 'html.parser')
        detail_chart_range = content.find('div' , {'class' : 'trader_detail_chart'})
        bars = content.find_all('div' , {'class' : 'bars'})[0].find_all('div' , {'class' : 'bar'})
        Date = []
        for bar in bars:
          date = bar.find('div' , {'class' : 'data_label'}).text.strip().replace(' ','').replace('\n' , ' ').split()
          date = ' '.join(date[1:3])
          result = {
              'Date':date
          }
          Date.append(result)

        Profitability = []
        for bar in bars:
          value = bar.find('div' , {'class' : 'data_value'}).text.strip().replace('%' , '')
          result = {
              'Profitability %' : value
          }
          Profitability.append(result)

        bars = content.find_all('div' , {'class' : 'bars'})[1].find_all('div' , {'class' : 'bar'})
        Risk = []
        for bar in bars:
          value = bar.find('div' , {'class' : 'data_value'}).text.strip().replace('risk' , '')
          result = {
              'Risk' : value
          }
          Risk.append(result)

        bars = content.find_all('div' , {'class' : 'bars'})[2].find_all('div' , {'class' : 'bar'})
        Copying = []
        for bar in bars:
          value = bar.find('div' , {'class' : 'data_value'}).text.strip()
          result = {
              'Copying' : value
          }
          Copying.append(result)

        df = pd.DataFrame({
            'Date': Date,
            'Risk': Risk,
            'Copying': Copying,
            'Profitability %': Profitability
        })

        df['Date'] = df['Date'].apply(lambda x: x['Date'])
        df['Risk'] = df['Risk'].apply(lambda x: x['Risk'])
        df['Copying'] = df['Copying'].apply(lambda x: x['Copying'])
        df['Profitability %'] = df['Profitability %'].apply(lambda x: x['Profitability %'])

        df.set_index('Date', inplace=True)

        csv_file = f"{path}/annual rates.csv"
        df.to_csv(csv_file)

In [5]:
import requests
from bs4 import BeautifulSoup
import csv

def popular_tools(id:int , path:str):
    url_trader = f'https://my.litefinance.org/traders/info?id={id}'
    r = requests.get(url_trader)
    if r.status_code == 200:
        content = BeautifulSoup(r.text, 'html.parser')
        trader_detail_tools = content.find_all('div', {'class': 'panel_inner'})[2]
        content_row = trader_detail_tools.find_all('div', {'class': 'content_row_inner'})
        detail_tools = []
        for row in content_row:
          profitable_tag = row.find_all('div', {'class': 'data'})[2].find('div', 'data_value')
          profitable = profitable_tag.text.strip().replace('%', '')
          if float(profitable) >= 70:
            title = row.find('div', {'class': 'title'})
            trades = row.find_all('div', {'class': 'data'})[1].find('div', 'data_value')
            result = {
                'title': title.text.strip().replace('_o', '')[:6],
                'trades': trades.text.strip(),
                'Profitable %': profitable
            }
            detail_tools.append(result)

        field_names = ['title', 'trades', 'Profitable %']
        with open(f'{path}/popular tools.csv', mode='w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=field_names)
            writer.writeheader()
            for detail in detail_tools:
                writer.writerow(detail)

In [6]:
def Portfolio_composition(id:int , path:str):
    url_trader = f'https://my.litefinance.org/traders/info?id={id}'
    r = requests.get(url_trader)
    if r.status_code == 200:
        content = BeautifulSoup(r.text, 'html.parser')
        trader_detail = content.find('div' , {'class' : 'trader_detail_tools_chart_data'}).find_all('div' , {'class' : 'data'})
        Detail_name = []
        Detail_value = []
        for detail in trader_detail:
          mareket = detail.find('div' , {'class' : 'data_label'})
          precentage = detail.find('div' , {'class' : 'data_value'})
          Detail_name.append(f'{mareket.text.strip()} %')
          Detail_value.append(precentage.text.strip().replace('%', ''))

        with open(f'{path}/Portfolio composition.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(Detail_name)
            writer.writerow(Detail_value)

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import csv
import time

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)

def trading_history(id:int , path:str , currency : list):
  url = f'https://my.litefinance.org/traders/trades-history?id={id}'
  browser.get(url)
  for _ in range(1):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
  content = BeautifulSoup(browser.page_source , 'html.parser')
  rows = content.find_all('div' , {'class' : 'content_row'})
  history = []
  for row in rows:
    title = row.find_all('div' , {'class' : 'content_col'})[0].find('div' , {'class' : 'title'})
    title = title.text.strip()
    for symbol in currency:
      if symbol == title:
        opening_time = row.find_all('div' , {'class' : 'content_col'})[1].find('div' , {'class' : 'data'}).find('div' , {'class' : 'data_value'})
        closing_time = row.find_all('div' , {'class' : 'content_col'})[2].find('div' , {'class' : 'data'}).find('div' , {'class' : 'data_value'})
        action = row.find_all('div' , {'class' : 'content_col'})[3].find('div' , {'class' : 'label'})
        volume = row.find_all('div' , {'class' : 'content_col'})[4].find('div' , {'class' : 'data'}).find('div' , {'class' : 'data_value'})
        entry_point = row.find_all('div' , {'class' : 'content_col'})[5].find('div' , {'class' : 'data'}).find('div' , {'class' : 'data_value'})
        exit_point = row.find_all('div' , {'class' : 'content_col'})[6].find('div' , {'class' : 'data'}).find('div' , {'class' : 'data_value'})
        profit = row.find_all('div' , {'class' : 'content_col'})[7].find('div' , {'class' : 'data'}).find('div' , {'class' : 'data_value'})
        result = {
            'title' : title ,
            'opening time' : opening_time.text.strip(),
            'closing time' :closing_time.text.strip() ,
            'action' :action.text.strip() ,
            'volume' : volume.text.strip(),
            'entry point' : entry_point.text.strip() ,
            'exit point' : exit_point.text.strip() ,
            'profit (USD)' : profit.text.strip().replace(' USD','')
        }
        history.append(result)
  field_names = ['title' , 'opening time' , 'closing time' , 'action' , 'volume' , 'entry point' , 'exit point' , 'profit (USD)']
  with open(f'{path}/history.csv', mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=field_names)
    writer.writeheader()
    for hist in history:
        writer.writerow(hist)

In [8]:
import csv
def Get_info_for_trader(id:int , path:str):
  url_trader = f'https://my.litefinance.org/traders/info?id={id}'
  r = requests.get(url_trader)
  if r.status_code == 200:
    content = BeautifulSoup(r.text , 'html.parser')
    tradre_name = content.find('div' , {'class' : 'title'})
    panel_inner = content.find_all('div' , {'class' : 'panel_inner'})[0]
    country = panel_inner.find_all('div' , {'class' : 'data'})[0].find('div' , {'class' : 'data_value'})
    in_ranking = panel_inner.find_all('div' , {'class' : 'data'})[1].find('div' , {'class' : 'data_value'})
    personal_assets = panel_inner.find_all('div' , {'class' : 'data'})[2].find('div' , {'class' : 'data_value'})
    copy_traders_assets = panel_inner.find_all('div' , {'class' : 'data'})[3].find('div' , {'class' : 'data_value'})
    number_of_copy_traders = panel_inner.find_all('div' , {'class' : 'data'})[4].find('div' , {'class' : 'data_value'})
    risk = panel_inner.find_all('div' , {'class' : 'data'})[5].find('span')
    total_trades = content.find('div' , {'class' , 'trader_detail_tools_summary'}).find('div' , {'class' : 'panel_inner'}).find_all('div' , {'class' : 'data_value'})[0]
    profitability = content.find('div' , {'class' , 'intro'}).find('div' , {'class' : 'data_value'})
    result =  {
        'trader name' : tradre_name.text.strip() ,
        'id' : id ,
        'country' : country.text.strip() ,
        'in ranking(days)' : in_ranking.text.strip().replace('days',''),
        'personal assets (USD)': personal_assets.text.strip().replace('USD','').replace('~','').replace(' ',''),
        'copy traders assets (USD)':copy_traders_assets.text.strip().replace('USD','').replace('~','').replace(' ',''),
        'number of copy traders' : number_of_copy_traders.text.strip(),
        'risk' : risk.text.strip().replace('risk','') ,
        'total trades' :total_trades.text.strip() ,
        'profitability %' : profitability.text.strip().replace(' %' , '')
    }
    field_names = ['trader name', 'id', 'country' , 'in ranking(days)' , 'personal assets (USD)' , 'copy traders assets (USD)' , 'number of copy traders' , 'risk' , 'total trades' , 'profitability %']
    with open(f'{path}/Info about trader.csv', mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=field_names)
        writer.writeheader()
        writer.writerow(result)


In [9]:
def List_currency(id:int , path:str):
  return pd.read_csv(path)['title'].values.tolist()

In [27]:
import pandas as pd
import os

traders = pd.read_csv('traders.csv')

for index, row in traders.iterrows():
    trader_name = row['trader name']
    trader_id = row['id']
    new_folder_path = f'/content/traders info/{trader_id}'
    try:
        os.makedirs(new_folder_path, exist_ok=True)
    except OSError as e:
        print(f"Error creating directory: {e}")

    popular_tools(id=trader_id, path=new_folder_path)

    popular_tools_path = f'{new_folder_path}/popular tools.csv'
    try :
      list_currency = List_currency(id = trader_id , path = popular_tools_path)
    except:
      list_currency = ''

    if list_currency:
      trading_history(id=trader_id, path=new_folder_path , currency = list_currency )
      Get_info_for_trader(id=trader_id, path=new_folder_path)
      annual_rates(id=trader_id, path=new_folder_path)
      Portfolio_composition(id=trader_id, path=new_folder_path)




In [ ]:
folder_path = '/content/traders info'
files_name = os.listdir(folder_path)
id_traders = []

for id in files_name:


In [11]:
import pandas as pd
import numpy as np
import os

folder_path = '/content/traders info'
files_name = os.listdir(folder_path)
id_traders = []

#for id in files_name:
#  popular_tools = f'{folder_path}/{id}/popular tools.csv'
#  list_currency = List_currency(id = id , path = popular_tools)
#  if list_currency:
#    print(list_currency)


print(files_name)


['953272', '1003663', '1177279', '1850567', '1582212', '1840700', '2219194', '1411219', '2219860', '1219691', '968181', '2242731', '2826138', '671915', '2290559', '780927', '858979', '753248', '1679829', '1120932', '941691', '1256561', '1216727', '1315262', '956079', '2283539', '1022035', '1184848', '1395992', '964460', '9490', '1263305', '2513359', '3172785', '1939365', '2210431', '1215528', '2040949', '1683067', '899372', '1120915', '2524087', '1150932', '1167765', '1240727', '2100953', '2226687', '1441362', '471387', '913896', '1316754', '1976638', '1273817', '1152404', '1534720', '1216657', '2767461', '1120958']
